This code is used to trimmed and chunk the data into smaller pieces.

In [ ]:
import os
import pandas as pd

In [ ]:
def retrun_individuals(file):
    data = pd.read_csv(file)
    return data[["individual"]]

def convert_naming_scheme(data):
    # if the value of an index of the list is M then change it to 1, else 2
    data = [1 if x == ""M"" else 2 for x in data]
    return data

def convert_str_to_list(data):
    # Remove brackets and split by spaces
    data = data.replace("[", "").replace("]", "")
    data_list = data.split()
    
    if (data_list[0] == ""M"" or data_list[0] == ""B""):
        data_list = convert_naming_scheme(data_list)

    # Convert string to int
    data_list = [int(i) for i in data_list]

    return data_list

def read_and_process_file(file):
    retruned_value = retrun_individuals(file)
    retruned_value["individual"] = retruned_value["individual"].apply(convert_str_to_list)
    return retruned_value

# list all the files in population-storage folder
files = os.listdir("./")
# sort the files in ascending order
files.sort()
# sort the files that start wth all_individuals
sub_list_of_files = [file for file in files if file.endswith(".csv")]

if not os.path.exists("non-trimmed"):
    os.makedirs("non-trimmed")

# read the files and process them
for file in sub_list_of_files:
        print("Reading ", file, " ...")
        data = read_and_process_file(file)
        print("Writing ", file, " ...")
        data.to_csv("non-trimmed/" + file, index=False)
        print("Done ", file, " ...")

In [ ]:
def trim_individuals(data):
    return data.replace(", ", " ")

# list all the files in non-trimmed folder
files = os.listdir("non-trimmed")
# sort the files in ascending order
files.sort()
# sort the files that start wth all_individuals
sub_list_of_files = [file for file in files if file.endswith(".csv")]

if not os.path.exists("trimmed"):
    os.makedirs("trimmed")

for file in sub_list_of_files:
    print("Reading ", "non-trimmed/" + file, " ...")
    data = pd.read_csv("non-trimmed/" + file)
    print("Trimming ", "non-trimmed/" + file, " ...")
    data["individual"] = data["individual"].apply(trim_individuals)
    print("Writing ", "non-trimmed/" + file, " ...")
    data.to_csv("trimmed/" + file, index=False)
    print("Done ", "trimmed/" + file, " ...")

In [ ]:
# list all the files in population-storage folder
files = os.listdir("trimmed")
# sort the files in ascending order
files.sort()
# sort the files that start wth all_individuals
sub_list_of_files = [file for file in files if file.endswith(".csv")]

data = pd.DataFrame(columns=["individual"])

for file in sub_list_of_files:
    print("Reading ", "trimmed/" + file, " ...")
    data = pd.concat([data, pd.read_csv("trimmed/" + file)])
    print("Done ", "trimmed/" + file, " ...")

print("Resetting Index ...")
data.reset_index(drop=True)
print("Dropping Duplicates ...")
data.drop_duplicates()
print("Data shape: ", data.shape)
rows = data.shape[0]
print("Rows: ", rows)
chunk_count = int(rows / 100000) + 1
print("Chunk count: ", chunk_count)

if not os.path.exists("chunks"):
    os.makedirs("chunks")

for i in range(chunk_count):
    print("Writing chunk ", i, " From: ", i*100000, " , To: ", (i+1)*100000, " ...")
    data.iloc[i*100000:(i+1)*100000].to_csv("chunks/data_chunked-1_"+str(i)+".csv", index=False)